In [40]:
import pandas, numpy

## functions & options


In [41]:
input_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7//0 in_silico/Python/1)data_input/'
output_file_directory = '/Users/kja11/OneDrive - Háskóli Íslands/PhD ATG7/0 in_silico/Python/3)output/'

# ATG7 expression, ensembl. Data preparation 

## 1) Normal tissue + Primary tumors selection, >20 samples

In [42]:
%%time
path = input_file_directory + "xenabrowser_brut_data/xena_surv_ATG7.tsv"
df_ori= pandas.read_csv(path, sep = "\t")
print(df_ori.shape)
df_ori.head(2)

(19131, 31)
Wall time: 92.8 ms


,sample,samples,_sample_type,_primary_site,ENSG00000197548.12,ENST00000451513.5,ENST00000435760.5,ENST00000451830.5,ENST00000460444.5,ENST00000470474.1,...,ENST00000434066.6,ENST00000478638.5,ENST00000461278.1,ENST00000460291.1,ENST00000467121.1,ENST00000414717.5,ENST00000427759.5,ENST00000446110.1,OS,OS.time
0,TARGET-20-PARUBT-40,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,4.401,-9.966,0.3685,2.656,-0.7108,-1.086,...,-9.966,-9.966,-0.6193,-5.012,-9.966,0.8568,-9.966,-9.9660,NaN,NaN
1,TARGET-20-PATJHJ-40,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White blood cell,2.820,-9.966,-0.1345,-1.595,-2.9320,-9.966,...,-9.966,-9.966,-9.9660,-3.308,-9.966,-2.0530,-2.727,-0.2328,NaN,NaN


In [43]:
# Column selection
df = df_ori[['sample',
            '_sample_type',
            '_primary_site',
             'OS', 'OS.time',
            'ENSG00000197548.12',
            'ENST00000354449.7', 
            'ENST00000354956.9',
            'ENST00000446450.6']]

# We have Adrenal Gland and Adrenal gland. I need to put "G".
df = df.assign(_primary_site=df['_primary_site'].str.title())

# Column rename
df = df.rename(columns = {'sample' : 'sample',
                    '_sample_type' : 'Sample_Type',
                    'OS.time' : 'OS_time',
                    '_primary_site' : 'Primary_Site',
                    'ENSG00000197548.12': 'ATG7',
                    'ENST00000354449.7': 'ATG7_1',
                    'ENST00000354956.9': 'ATG7_2',
                    'ENST00000446450.6': 'ATG7_3'})
df.head(2)

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,4.401,1.390,1.522,-1.732
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,2.820,1.118,-1.086,-9.966


In [44]:
%%time
# transform data from log2(tpm + 0.001) to tpm
print(f'min in log2(tpm+0.001): {df.iloc[:,5:].min().min()}')
#to tpm
df.iloc[:,5:] = ((2**df.iloc[:,5:]) - 0.001)
print(f'min in tpm: {df.iloc[:,5:].min().min()}')

# if value is negative, it's zero
df.iloc[:,5:] = (df.iloc[:,5:]).where(df.iloc[:,5:] > 0, 0)
print(f'new min in tpm: {df.iloc[:,5:].min().min()}')

# save tpm in a separate df
df_tpm = df.iloc[:,5:]

min in log2(tpm+0.001): -9.966
min in tpm: -1.4951130034891735e-07
new min in tpm: 0.0
Wall time: 10 ms


In [45]:
%%time
# transform data from tpm to log2(tpm + 1)
#to tpm +1
df.iloc[:,5:] = df.iloc[:,5:] + 1
print(f'min in tpm+1: {df.iloc[:,5:].min().min()}')

#to log2(tpm+1)
df.iloc[:,5:] = numpy.log2(df.iloc[:,5:]) 
print(f'min in log2(tpm+1): {df.iloc[:,5:].min().min()}')

df.head(2)

min in tpm+1: 1.0
min in log2(tpm+1): 0.0
Wall time: 6.92 ms


,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,4.467655,1.855905,1.952665,0.37855
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,3.010892,1.664236,0.555881,0.00000


In [46]:
#Prepare the columns with TPM, percentage, normalized
# Column rename
df_tpm = df_tpm.rename(columns = {'ATG7' : 'ATG7_tpm',
                                  'ATG7_1' : 'ATG7_1tpm',
                                  'ATG7_2' : 'ATG7_2tpm',
                                  'ATG7_3' : 'ATG7_3tpm'})
 
# Calcul to see percentage of expression of the 3 isoforms of ATG7 
df_tpm = df_tpm.assign(ATG7_prot_tot= df_tpm['ATG7_1tpm'] + df_tpm['ATG7_2tpm'] + df_tpm['ATG7_3tpm'])

df_tpm = df_tpm.assign(ATG7_1perc= (df_tpm['ATG7_1tpm'] *100)/ df_tpm['ATG7_prot_tot'],
              ATG7_2perc= (df_tpm['ATG7_2tpm'] *100)/ df_tpm['ATG7_prot_tot'],
              ATG7_3perc= (df_tpm['ATG7_3tpm'] *100)/ df_tpm['ATG7_prot_tot'])                              

#Add columns, normalized on total
df_tpm = df_tpm.assign(ATG7_1norm= df_tpm['ATG7_1tpm']/df_tpm["ATG7_tpm"],
                       ATG7_2norm= df_tpm['ATG7_2tpm']/df_tpm["ATG7_tpm"])
df_tpm.head(2)

,ATG7_tpm,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,21.125765,2.619787,2.870889,0.300034,5.790710,45.241201,49.577495,5.181305,0.124009,0.135895
1,7.060624,2.169459,0.470066,0.000000,2.639524,82.191275,17.808725,0.000000,0.307262,0.066576


In [47]:
df = pandas.concat([df, df_tpm], axis=1)
df.head(2)

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_tpm,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
0,TARGET-20-PARUBT-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,4.467655,1.855905,1.952665,0.37855,21.125765,2.619787,2.870889,0.300034,5.790710,45.241201,49.577495,5.181305,0.124009,0.135895
1,TARGET-20-PATJHJ-40,Recurrent Blood Derived Cancer - Peripheral Blood,White Blood Cell,NaN,NaN,3.010892,1.664236,0.555881,0.00000,7.060624,2.169459,0.470066,0.000000,2.639524,82.191275,17.808725,0.000000,0.307262,0.066576


In [48]:
#what kind of samples are the data ? 
df['Sample_Type'].value_counts()

Primary Tumor                                        9185
Normal Tissue                                        7429
Solid Tissue Normal                                   738
Cell Line                                             433
Metastatic                                            393
Primary Solid Tumor                                   286
Primary Blood Derived Cancer - Peripheral Blood       239
Primary Blood Derived Cancer - Bone Marrow            237
Recurrent Blood Derived Cancer - Bone Marrow          104
Recurrent Tumor                                        45
Recurrent Solid Tumor                                  13
Post treatment Blood Cancer - Bone Marrow              12
Additional - New Primary                               11
Recurrent Blood Derived Cancer - Peripheral Blood       3
Post treatment Blood Cancer - Blood                     1
Control Analyte                                         1
Additional Metastatic                                   1
Name: Sample_T

In [49]:
#what tissues are in the data ? 
df['Primary_Site'].value_counts()

Brain                         1846
Lung                          1410
Breast                        1391
Skin                          1282
Kidney                        1193
Esophagus                      848
Prostate                       648
Colon                          639
Stomach                        624
Blood Vessel                   606
White Blood Cell               595
Thyroid Gland                  571
Head And Neck Region           564
Liver                          531
Adipose Tissue                 515
Ovary                          515
Blood                          444
Bladder                        435
Muscle                         396
Heart                          377
Pancreas                       350
Testis                         319
Cervix                         309
Thyroid                        279
Nerve                          278
Soft Tissue,Bone               264
Adrenal Gland                  205
Endometrium                    204
Paraganglia         

In [50]:
#Download the filtered sample list
path = "samples_IDs/filtered_samples_list"

samples_list_normal= numpy.loadtxt(input_file_directory+path+'_normal.tsv', delimiter="\t", dtype=str)
samples_list_primary= numpy.loadtxt(input_file_directory+path+'_primary.tsv', delimiter="\t", dtype=str)

print(len(samples_list_normal), len(samples_list_primary))

5903 8022


In [51]:
#Creation of two dataframe, Normal tissue and Primary tumor.
df_ATG7_Normal_all = df.loc[df['sample'].isin(samples_list_normal)]
df_ATG7_Primary_all = df.loc[df['sample'].isin(samples_list_primary)]

print('normal_tissue' , df_ATG7_Normal_all.shape)
print('primary_tumors' , df_ATG7_Primary_all.shape)

normal_tissue (5903, 19)
primary_tumors (8022, 19)


In [52]:
#How many samples represent each organs?
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
infos = 'normal_tissue', 'primary_tumor'

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    
    print(info.upper())
    print(df["Primary_Site"].value_counts())
    print('How many samples without site info? :', df["Primary_Site"].isna().sum())
    print('___________________________________')

NORMAL_TISSUE
Brain              818
Esophagus          629
Blood Vessel       603
Skin               544
Adipose Tissue     510
Colon              295
Lung               285
Thyroid            278
Nerve              278
Breast             179
Testis             157
Muscle             150
Heart              122
Adrenal Gland      121
Stomach            119
Pituitary          107
Prostate            99
Spleen              97
Small Intestine     90
Ovary               88
Vagina              84
Uterus              78
Salivary Gland      53
Pancreas            36
Liver               27
Kidney              15
Blood               13
Cervix Uteri        10
Bladder              9
Fallopian Tube       5
Name: Primary_Site, dtype: int64
How many samples without site info? : 4
___________________________________
PRIMARY_TUMOR
Breast                     1005
Lung                        958
Kidney                      761
Brain                       637
Head And Neck Region        477
Thyroid Gland

In [53]:
#remove NaN in primary site
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
for df in dfs: 
#     df['Primary_Site'].dropna(axis=0)
    df.dropna(subset=['Primary_Site'], inplace=True)
df_ATG7_Normal_all.tail(2)

,sample,Sample_Type,Primary_Site,OS,OS_time,ATG7,ATG7_1,ATG7_2,ATG7_3,ATG7_tpm,ATG7_1tpm,ATG7_2tpm,ATG7_3tpm,ATG7_prot_tot,ATG7_1perc,ATG7_2perc,ATG7_3perc,ATG7_1norm,ATG7_2norm
8079,GTEX-ZV6S-1526-SM-5NQ7O,Normal Tissue,Uterus,NaN,NaN,3.324552,2.985517,0.16349,0.0,9.018203,6.920094,0.119993,0.0,7.040087,98.29557,1.70443,0.0,0.767347,0.013306
8080,GTEX-WWYW-2826-SM-4MVOQ,Normal Tissue,Uterus,NaN,NaN,3.282308,2.790908,0.00000,0.0,8.729111,5.920653,0.000000,0.0,5.920653,100.00000,0.00000,0.0,0.678265,0.000000


In [54]:
#Remove tissues with lower than 20 samples. 
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
infos = 'Normal:', 'Primary:'

df_ATG7_filtered_list = []

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    print(info)
    
    #delete samples without site info
    df.dropna(subset=['Primary_Site'], inplace=True)
    
    element_to_delete = []
    #select all the elements in primary sites, and note which one are <20
    working_elements = df["Primary_Site"].to_list()
    for element in working_elements:
        if working_elements.count(element) < 20:
            element_to_delete.append(element)
            
    #Convert to Set() to have a clear list because it cannot contain duplicate. 
    element_to_delete = set(element_to_delete)
    print('elements with less than 20 samples are:', element_to_delete)
    
    #Remove the samples where the tissue is represented less than 20 times
    print('shape with all the elements', df.shape)
    for element in element_to_delete:
        df = df.loc[df["Primary_Site"] != element]
    print('shape after removing the elements', df.shape)
    
    df_ATG7_filtered_list.append(df)
    print()
    
df_ATG7_Normal_all = df_ATG7_filtered_list[0]
df_ATG7_Primary_all = df_ATG7_filtered_list[1]

Normal:
elements with less than 20 samples are: {'Bladder', 'Blood', 'Kidney', 'Cervix Uteri', 'Fallopian Tube'}
shape with all the elements (5899, 19)
shape after removing the elements (5847, 19)

Primary:
elements with less than 20 samples are: set()
shape with all the elements (8022, 19)
shape after removing the elements (8022, 19)



In [55]:
#all tissues save to csv
path = 'ATG7/TCGA_GTEX/'
path2 = 'all_tissues/dataframes_for_input/'
df_ATG7_Normal_all.to_csv(output_file_directory+path+path2+'df_ATG7_Normal_all.csv',sep = ",", index=False)
df_ATG7_Primary_all.to_csv(output_file_directory+path+path2+'df_ATG7_Primary_all.csv',sep = ",", index=False)

## 2) Filtered data: >20 samples and present in the two groups       (normal tissue & primary tumors)

In [56]:
#How many samples represent each organs?
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
infos = 'normal_tissue', 'primary_tumor'

for i in range(len(dfs)):
    df = dfs[i]
    info = infos[i]
    
    print(info.upper())
    print(df["Primary_Site"].value_counts())
    print('___________________________________')

NORMAL_TISSUE
Brain              818
Esophagus          629
Blood Vessel       603
Skin               544
Adipose Tissue     510
Colon              295
Lung               285
Thyroid            278
Nerve              278
Breast             179
Testis             157
Muscle             150
Heart              122
Adrenal Gland      121
Stomach            119
Pituitary          107
Prostate            99
Spleen              97
Small Intestine     90
Ovary               88
Vagina              84
Uterus              78
Salivary Gland      53
Pancreas            36
Liver               27
Name: Primary_Site, dtype: int64
___________________________________
PRIMARY_TUMOR
Breast                     1005
Lung                        958
Kidney                      761
Brain                       637
Head And Neck Region        477
Thyroid Gland               464
Prostate                    412
Stomach                     397
Bladder                     339
Ovary                       325
Colon   

In [57]:
#Change names to regroup Thyroid and Thyroid Gland
df_ATG7_Primary_all['Primary_Site'].replace('Thyroid Gland', 'Thyroid', inplace=True)
#to see what tissues are present in both group
norm_sites = df_ATG7_Normal_all['Primary_Site']
prim_sites = df_ATG7_Primary_all['Primary_Site']

set1 = set(norm_sites)
set2 = set(prim_sites)
inter_site = set1.intersection(set2)
print('tissues in the two groups:')
inter_site

tissues in the two groups:


{'Adrenal Gland',
 'Brain',
 'Breast',
 'Colon',
 'Esophagus',
 'Liver',
 'Lung',
 'Ovary',
 'Pancreas',
 'Prostate',
 'Skin',
 'Stomach',
 'Testis',
 'Thyroid',
 'Uterus'}

In [58]:
#Select only tissues present in the two groups
dfs = df_ATG7_Normal_all, df_ATG7_Primary_all
dfs_inter_L = []
for df in dfs:
    print(df.shape)
    df_inter = df[df['Primary_Site'].isin(inter_site)]
    dfs_inter_L.append(df_inter)
    print(df_inter.shape, '\n')
df_ATG7_Normal_filt = dfs_inter_L[0]
df_ATG7_Primary_filt = dfs_inter_L[1]
print(df_ATG7_Primary_filt["Primary_Site"].value_counts())

#save to csv
path2 = 'all_filtered_tissues/dataframes_for_input/'
df_ATG7_Normal_filt.to_csv(output_file_directory+path+path2+'df_ATG7_Normal_filtered.csv',sep = ",", index=False)
df_ATG7_Primary_filt.to_csv(output_file_directory+path+path2+'df_ATG7_Primary_filtered.csv',sep = ",", index=False)

(5847, 19)
(3753, 19) 

(8022, 19)
(5300, 19) 

Breast           1005
Lung              958
Brain             637
Thyroid           464
Prostate          412
Stomach           397
Ovary             325
Colon             283
Liver             180
Esophagus         177
Pancreas          162
Testis            126
Skin               77
Adrenal Gland      52
Uterus             45
Name: Primary_Site, dtype: int64
